# Copy Number Pipeline

In [2]:
from __future__ import print_function
import os.path
import numpy as np
import pandas as pd
from IPython.display import Image,display
import dalmatian as dm
from IPython.core.display import HTML 
from src.CCLE_postp_function import *
import sys
sys.path.insert(0, '../JKBio/')
import TerraFunction as terra
import rpy2
%load_ext autoreload
%autoreload 2
%load_ext rpy2.ipython
from taigapy import TaigaClient
tc = TaigaClient()
from gsheets import Sheets
sheets = Sheets.from_files('~/.client_secret.json', '~/.storage.json')
replace = {'T': 'Tumor', 'N': 'Normal', 'm': 'Unknown', 'L': 'Unknown'}

In [3]:
samplesetname = "19Q4"
prevname = '19Q3'
prevprevname ='19Q2'
virtual_public=''
virtual_dmc=''
virtual_internal=''

workspace1="broad-genomics-delivery/Getz_IBM_CellLines_Exomes"
workspace2="broad-firecloud-ccle/CCLE_DepMap_WES"
workspace3="broad-genomics-delivery/CCLE_DepMap_WES"
refworkspace="broad-firecloud-ccle/DepMap_WES_CN_hg38"
source1="ibm"
source2="ccle"
source3="ccle"
release = samplesetname
sheeturl = "https://docs.google.com/spreadsheets/d/115TUgA1t_mD32SnWAGpW9OKmJ2W5WYAOs3SuSdedpX4"

In [4]:
%%R
release <- '19Q4'
prevname <- '19Q3'

genome_version <- 'hg38'
hg38_cyto_band_reference <- '../JKBio/data/hg38_cytoband.gz'
new_samples_copy_number_broad_wes <- 'temp/cnv_ccle.called.seg'

In [5]:
wm1 = dm.WorkspaceManager(workspace1)
wm2 = dm.WorkspaceManager(workspace2)
wm3 = dm.WorkspaceManager(workspace3)

refwm = dm.WorkspaceManager(refworkspace)

In [ ]:
extract_to_change = {'ref_bams':'WES_bam'}

In [ ]:
newsample = createDatasetWithNewCellLines(refwm, samplesetname, wmfroms = [wm1, wm2, wm3], sources=[source1, source2, source3], gsfolderto='gs://ccle_bams/wes/', match='ACH', participantslicepos=10, accept_unknowntypes=True, extract = extract_to_change, dry_run = True)

## Check that we have all the cell lines we expect for this release
This involves comparing to the list in the Google sheet "Cell Line Profiling Status."

In [ ]:
# this function may not work - it hasn't been tested
url = 'https://docs.google.com/spreadsheets/d/1qus-9TKzqzwUMNWp8S1QP4s4-3SsMo2vuQRZrNXf7ag/edit?ts=5db85e27#gid=0&fvid=1627883727'

compareToCuratedGS(url, sample = newsample[0], samplesetname = samplesetname, colname = 'CN New to internal')

# run Terra pipeline

In [ ]:
gsheet.index = gsheet['DepMap ID']
new_cn = gsheet[gsheet['CN New to internal'] == '19Q4tent']

In [ ]:
terra_participants = refwm.get_samples()['participant'].tolist()
terra_participants

In [ ]:
set(new_cn.index.tolist()) - set(terra_participants)
## NOTE: we do not want to include ACH-002446-cas9 in this release

In [ ]:
bamtoubam= "BamToUnmappedRGBams_MC"
ubamtofilelist = "Generate_uBAM_File_List"
realign="Realign_WES_GATK4"

In [ ]:
subid = refwm.create_submission(bamtoubam,samplesetname,"sample_set","this.samples")
terra.waitForSubmission(refworkspace, subid)

In [ ]:
subid = refwm.create_submission(ubamtofilelist,samplesetname,"sample_set","this.samples")
terra.waitForSubmission(refworkspace, subid)

In [ ]:
subid = refwm.create_submission(realign,samplesetname,"sample_set","this.samples")
terra.waitForSubmission(refworkspace, subid)

In [ ]:
submission_id= refwm.create_submission("CNV_sample_XX",etype='sample_set',entity=samplesetname,expression='this.samples')
terra.waitForSubmission(refworkspace,submission_id)

In [ ]:
submission_id = refwm.create_submission("Aggregate_CN_seg_files",entity=samplesetname)
terra.waitForSubmission(refworkspace,submission_id)

In [24]:
aggregated = refwm.get_entities('sample_set').loc[samplesetname]["combined_seg_file"]
aggregated

'gs://fc-secure-d2a2d895-a7af-4117-bdc7-652d7d268324/b904ecf9-8eb2-4fa1-be0b-3749b7b57364/aggregate_CN_segments_wrkflw/96d5b974-e788-4755-9a58-a99f8a904500/call-aggregate_CN_segments/19Q4.called.seg'

### Save the workflow configurations used

In [ ]:
terra.saveConfigs(refworkspace,'data/'+sample_set_id+'/CNVconfig')

## deleting unmapped bams

In [25]:
sam = refwm.get_samples()
samplesinset = sam[sam.index.isin(refwm.get_sample_sets().loc[sample_set_id].samples)]

NameError: name 'sample_set_id' is not defined

In [ ]:
for val in samplesinset.readgroup_ubams:
    ubams = ''
    for v in val:
        ubams+=' '+v
    os.system('gsutil -m rm'+ubams)

In [ ]:
onlycol = ['hg38_analysis_ready_bam',
'hg38_analysis_ready_bam_index',
'hg38_analysis_ready_bam_md5']
newgs = 'gs://ccle_bams/hg38_wes/'
for k, val in samplesinset[onlycol].iterrows():
    data = ''
    for v in val:
        data+=' '+v
    ! gsutil -m mv $data $newgs

changeGSlocation(refwm,prevgslist=['gs://fc-secure-d2a2d895-a7af-4117-bdc7-652d7d268324/'], newgs=newgs, onlycol=onlycol, entity='samples')

In [26]:
! gsutil cp $aggregated "temp/cnv_ccle.called.seg"

Copying gs://fc-secure-d2a2d895-a7af-4117-bdc7-652d7d268324/b904ecf9-8eb2-4fa1-be0b-3749b7b57364/aggregate_CN_segments_wrkflw/96d5b974-e788-4755-9a58-a99f8a904500/call-aggregate_CN_segments/19Q4.called.seg...
/ [1 files][448.9 KiB/448.9 KiB]                                                
Operation completed over 1 objects/448.9 KiB.                                    


## If want to reprocess something

In [ ]:
#%%R
#segments_unfiltered <- readr::read_csv('temp/wes.19Q3.segmented.cn')

## post Procesing

In [69]:
%%R
source('../gkugener/RScripts/load_libraries_and_annotations.R')
source("src/CCLE_postp_function.R")
library('celllinemapr')
library('magrittr')
library('taigr')
library('cdsomics')
library('readr')

R[write to console]: Error in library(dplyr) : there is no package called ‘dplyr’
Calls: <Anonymous> ... withVisible -> source -> withVisible -> eval -> eval -> library




Error in library(dplyr) : there is no package called ‘dplyr’
Calls: <Anonymous> ... withVisible -> source -> withVisible -> eval -> eval -> library


In [ ]:
%%R
paste0("wes.",prevname,".segmented")

In [ ]:
%%R
# Previous release copy number profiles. This line will need to be updated as well
wes.priority.cn.seg.profiles <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.file=paste0("wes.",prevname,".segmented"),data.version='33') %>%
  dplyr::select(DepMap_ID, Chromosome, Start, End, Num_Probes, Segment_Mean, Source)

### getting blacklist

In [ ]:
%%R
segments <- processSegments(new_samples_copy_number_broad_wes)
segments <- filterForCCLE(segments)
segments <- interpolateGapsInSegmented(segments)$segs
segments <- extendEndsOfSegments(segments,hg38_cyto_band_reference)
print(segments)
print(head(wes.priority.cn.seg.profiles))
# reprioritize also undo logtransform
segments_unfiltered <- reprioritizeData(segments, wes.priority.cn.seg.profiles)
# Fill in the gaps on the entire dataset
# Extend start sites to 1, end sites to the end of the chromosome?

In [ ]:
%%R
write.table(segments_unfiltered, file = paste0("temp/wes.",release,".segmented.cn"), sep = ',', quote = F, row.names = F) 

In [ ]:
%%R
entrezgenes <- generateEntrezGenes()

In [ ]:
%%R
head(corner(entrezgenes))

In [ ]:
%%R
res <- generateGeneLevelMatrixFromSegments(entrezgenes, segments_unfiltered)

In [ ]:
%%R
genematrix_unfiltered <- res$gene_level_data_hg38
corner(genematrix_unfiltered)

In [ ]:
%%R
write.table(genematrix_unfiltered, file = paste0('temp/wes.',release,'.gene.cn'), 
sep = ',', quote = F, row.names = T)

## Validation step

In [32]:
wes_19Q3_gene = tc.get(name='segmented-cn-wes-prioritzed-7fe1', version=33, file='wes.19Q3.gene')

[##################]100% |  40.6 MiB/s | 539.1 MiB / 539.1 MiB | Time:  0:00:13


In [33]:
wes_19Q3_gene.values.max()

148.13492426

In [56]:
a = np.array([1.265172,1.141525,1.295836])

In [57]:
a

array([1.265172, 1.141525, 1.295836])

In [68]:
a

array([1.265172, 1.141525, 1.295836])

In [47]:
wes_19Q3_gene

,A1BG (1),NAT2 (10),ADA (100),CDH2 (1000),AKT3 (10000),GAGE12F (100008586),RNA5-8SN5 (100008587),RNA18SN5 (100008588),RNA28SN5 (100008589),LINC02584 (100009613),...,HNRNPDL (9987),DMTF1 (9988),PPP4R1 (9989),CDH1 (999),SLC12A6 (9990),PTBP3 (9991),KCNE2 (9992),DGCR2 (9993),CASP8AP2 (9994),SCO2 (9997)
ACH-000001,1.265172,1.141525,1.295836,0.662752,1.012791,2.048090,1.261577,1.261577,1.261577,1.001924,...,0.982285,0.936155,1.612348,0.680497,0.905825,0.672772,1.635826,1.261577,0.993110,0.678546
ACH-000002,1.013633,0.974174,1.017512,1.489327,1.012618,0.527652,1.030817,1.030817,1.030817,0.995799,...,0.998570,1.005580,1.489327,1.021939,0.981002,1.007272,1.009071,1.030817,1.002184,1.030817
ACH-000003,1.031898,1.095812,1.876833,0.536481,1.002498,1.008422,0.932645,0.932645,0.932645,1.633538,...,0.896888,1.167725,0.991923,1.258670,0.836420,1.025338,0.980575,0.184322,0.958799,0.184322
ACH-000004,1.349290,1.086509,0.785509,1.077061,1.086057,1.026334,1.049353,1.049353,1.049353,1.102287,...,0.767213,0.781653,1.045867,0.770678,1.124513,0.765991,1.984671,1.141159,1.122566,0.768331
ACH-000005,1.222033,1.162207,0.761254,1.072458,1.120937,0.437435,0.795196,0.795196,0.795196,1.142662,...,0.824491,0.767485,0.794864,0.757720,1.172731,0.769412,2.308558,0.795196,1.141036,0.756453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-001846,0.984617,1.072726,1.069769,1.004626,1.052576,0.527245,1.034755,1.034755,1.034755,1.085688,...,1.055702,1.601603,1.004626,1.071370,0.525225,1.048270,0.528907,1.055911,0.535124,1.001655
ACH-001858,0.829331,0.802194,1.258846,0.789034,1.074852,0.682834,0.815913,0.815913,0.815913,1.153987,...,0.690972,1.211705,0.693304,0.870386,0.989552,1.232196,0.793702,0.815913,1.166469,0.815913
ACH-001863,1.467836,0.683695,1.011538,1.345138,0.941841,0.646753,1.612338,1.612338,1.612338,1.513787,...,2.219316,1.345728,0.760783,1.114001,1.006464,0.708956,0.737090,1.646575,0.680696,0.669828
ACH-001864,1.032269,0.671964,2.067528,0.656176,1.321251,1.356937,1.288708,1.288708,1.288708,1.021389,...,0.720468,0.725135,0.656176,1.313839,1.302492,0.888370,0.965422,1.288708,1.051116,1.288708


In [54]:
genecn

,A1BG (1),NAT2 (10),ADA (100),CDH2 (1000),AKT3 (10000),GAGE12F (100008586),RNA5-8SN5 (100008587),RNA18SN5 (100008588),RNA28SN5 (100008589),LINC02584 (100009613),...,HNRNPDL (9987),DMTF1 (9988),PPP4R1 (9989),CDH1 (999),SLC12A6 (9990),PTBP3 (9991),KCNE2 (9992),DGCR2 (9993),CASP8AP2 (9994),SCO2 (9997)
ACH-000001,2.138670,2.072497,2.155488,1.841208,2.006410,2.617200,2.136709,2.136709,2.136709,2.000962,...,1.991170,1.968431,2.338422,1.849075,1.953681,1.845644,2.352661,2.136709,1.996559,1.848208
ACH-000002,2.006833,1.987145,2.008783,2.265311,2.006323,1.783072,2.015491,2.015491,2.015491,1.997901,...,1.999285,2.002793,2.265311,2.011011,1.990532,2.003640,2.004542,2.015491,2.001092,2.015491
ACH-000003,2.016037,2.048701,2.504030,1.786776,2.001250,2.004217,1.966716,1.966716,1.966716,2.351269,...,1.949365,2.086298,1.995967,2.135125,1.920527,2.012725,1.990320,1.649056,1.979546,1.649056
ACH-000004,2.185190,2.043903,1.896737,2.039045,2.043670,2.013227,2.024887,2.024887,2.024887,2.052050,...,1.888297,1.894953,2.023116,1.889891,2.063599,1.887735,2.574761,2.072305,2.062584,1.888811
ACH-000005,2.115284,2.083382,1.885560,2.036684,2.061735,1.745966,1.901229,1.901229,1.901229,2.073094,...,1.914913,1.888422,1.901075,1.883940,2.088949,1.889308,2.797818,1.901229,2.072241,1.883360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-002476,0.974170,1.006780,1.018446,0.970864,0.999301,2.138328,1.031839,1.031839,1.031839,1.024468,...,1.023400,1.027562,0.970864,1.013489,1.030947,1.029268,0.984310,1.031839,1.016296,1.031839
ACH-002508,0.790660,1.246051,1.270051,0.755437,0.991930,0.428101,1.037863,1.037863,1.037863,1.261410,...,0.773155,0.778138,0.755437,0.773553,1.019839,1.178633,0.937277,1.037863,1.028239,1.052359
ACH-002509,0.887740,1.075906,1.092301,0.849194,1.199473,0.908285,1.095349,1.095349,1.095349,1.090689,...,1.062138,1.100696,0.849194,1.095584,0.648855,0.897010,0.878599,1.095349,1.019189,1.095349
ACH-002510,1.054755,0.960984,0.961169,1.134161,1.506412,0.392116,1.160694,1.160694,1.160694,1.138698,...,0.713536,1.149565,1.134161,0.694136,0.992001,0.704184,1.141474,1.160694,1.141887,1.160694


In [34]:
wes_19Q3_gene.values.mean()

1.0261536239625233

In [35]:
wes_19Q3_gene.values.min()

1.927306e-10

In [9]:
genecn=wes_19Q4_gene

In [30]:
genecn.values.min()

148.13492426

In [29]:
2**(1.06425452573e-09)-1

7.376850241769262e-10

In [46]:
np.log2(2**np.log2((2**genecn.values)-1)-1).mean()

/home/jeremie/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log2
  """Entry point for launching an IPython kernel.


nan

In [6]:
genecn = pd.read_csv('temp/wes.'+release+'.gene.cn', sep = ',')

FileNotFoundError: [Errno 2] File b'temp/wes.19Q4.gene.cn' does not exist: b'temp/wes.19Q4.gene.cn'

In [ ]:
segmentcn = pd.read_csv('temp/wes.'+release+'.segmented.cn', sep = ',')

In [ ]:
genecn.index = [i[-1] for i in genecn.index.str.split('_').tolist()]
segmentcn["DepMap_ID"] = [i[-1] for i in segmentcn["DepMap_ID"].str.split('_').tolist()]

In [ ]:
prevprevname

In [ ]:
prev = set(tc.get(name='segmented-cn-wes-prioritzed-7fe1', file='wes.'+prevname+'.gene', version="33").index.tolist())
prevprev= set(tc.get(name='segmented-cn-wes-prioritzed-7fe1', file='CCLE_internal_19q2_gene_cn', version='25').index.tolist())

In [ ]:
new1 = set(genecn.index.values.tolist())
new2 = set(segmentcn['Sample'].values.tolist())
print(len(prev), len(prev & new1), len(new1), len(new1 & new2))

In [ ]:
checkAmountOfSegments(segmentcn,thresh = 750)

In [ ]:
checkGeneChangeAccrossAll(genecn, thresh=1.5)

In [ ]:
segmentcn.Start = segmentcn.Start.astype(int)
segmentcn.End = segmentcn.End.astype(int)

In [ ]:
if(genecn.values.max() > 100):
    print("TOO HIGH, not LOG2 transformed!")
if(len(genecn.index.tolist()) > len(set(genecn.index))):
    print("Duplicate CL, not reprioritized well!")

In [ ]:
segmentcn[segmentcn["DepMap_ID"]=="ACH-000658"]

In [ ]:
for k, val in samplesinset.iterrows():
    plot = val["modeled_segments_plot_tumor"]
    ! gsutil cp $plot temp/
    print(k)
    display(Image('temp/'+plot.split('/')[-1]))

These looks bad : 
M140325
OCIP5X

In [ ]:
gsheets = sheets.get(sheeturl).sheets[6].to_frame()
wes_dmc_embargo = [i for i in gsheets['WES_DMC_embargo'].values.tolist() if i is not np.nan]
blacklist = [i for i in gsheets['blacklist'].values.tolist() if i is not np.nan]

# Upload to taiga

## we Push full dataset in depmap taiga CN

In [7]:
## for genecn removing first blacklisted, then embargoed, to create two datasets
genecn = genecn.apply(lambda x: np.log2(1+x))
segmentcn['Segment_Mean'] = segmentcn['Segment_Mean'].apply(lambda x: np.log2(1+x))
genecn.to_csv('temp/wes.'+release+'.gene.cn')
segmentcn.to_csv('temp/wes.'+release+'.segmented.cn')

NameError: name 'genecn' is not defined

In [ ]:
tc.update_dataset(dataset_permaname="segmented-cn-wes-prioritzed-7fe1", 
                  upload_file_path_dict={
                    'temp/wes.'+release+'.gene.cn': 'NumericMatrixCSV',
                    'temp/wes.'+release+'.segmented.cn': 'TableCSV'},
                  dataset_description=
"""
# Copy Number

Combined segment and gene-level CN calls from Broad WES, Sanger WES, and Broad SNP. Relative CN, log2(x+1) transformed.

PORTAL TEAM SHOULD NOT USE THIS: There are lines here that should not make it even to internal. Must use subsetted dataset instead. These data will not make it on the portal starting 19Q1. With the DMC portal, there is new cell line release prioritization as to which lines can be included, so a new taiga dataset will be created containing CN for the portal.

These data are generated for Achilles to pull from to run CERES.

Versions to use:

v25 for 19Q2 (hg38 aligned, Broad WES and Sanger WES based calls were generated from bam realignment. SNP based calls are still from liftover). The gene mapping script was updated to improve the gene level matrix (to remove NAs). The segmented level matrix is untransformed relative CN. Gene level matrix is log2(CN + 1).
v20 for 19Q1 (version 21 is hg19)
v18: for 18Q4
v15: for 18Q3
v11: for 18Q2
Gene-level matrix in versions below 10 were using hg38 and not hg19. Version 11 is corrected and should be used instead

Calls on X, Y chromosome for profiles should not be used.

Prioritization is as follows:

Broad WES kept over everything
Sanger WES kept if:
This cell line did not fail fingerprinting
This cell line has no other CN data
This cell lines does not have CRISPR LFC data from the Achilles screen
This CN profile correlates better with Achilles CRISPR LFC data than Broad SNP CN OR the % gene-level difference between this cell lines CN profile from Sanger WES and Broad SNP < 2.5%
Broad SNP used for remaining lines with no Broad WES or with Sanger WES that does not pass the criteria above
The 'Source' column indicates which CN profile was used for that cell line.

version 6: renamed Sample column to CCLE_name for consistency for the Achilles pipeline
version 7: missing chordoma lines
version 8: fixed to names of two chordoma lines (changed suffix from CHORDOMA -> BONE) and removed renamed cell lines that were duplicated (with different names). Reran comparison using 18q2 LFC results. Gene level matrix will be generated for version 9
version 11: corrected error in gene-level matrix calculation (previously had been aligned to hg38 however alignment should be hg19). Segment level calls are unaffected.
versions 12-14: Sanger WES were multiplied by 2 so should not be used
version 15: internal segments and gene level matrices for 18q3 release including public version (removed black list lines and Broad WES < 6 months old). Gene level matrices are indexed using Broad IDs.
version 16: internal segments and gene level matrices for 18q4 release including public version (uses all SNP and only WES if those lines are present in the 18Q4 public Achilles dataset)
version 17: same as version 16 but with two additional line in the internal version
version 18: same as version 17 but switched one line in public to use SNP instead of WES because not in public Avana
version 20: two major changes occurred (1) we are using a FireCloud based pipeline for CN calling now for Broad WES data (2) we have moved to use hg38. This is accomplished by lifting over coordinates from hg19 to hg38 after processed by the CN pipeline.
v21 same as version 20, but we are using the original hg19 coordinates, not hg38
version 25: Broad WES and Sanger WES were realigned to hg38. SNP still uses liftover from hg19 to hg38

## ** Internal 19Q4****

Adding 17 new cell lines, 3 reprioritized cell lines. log2(COPY RATIO+1). 
Some cells lines have been flagged as:

 - having bad looking copy ration plots = 
 - having too many segments = [ ACH-000458 762, ACH-000848 1171, ACH-001017 1223, ACH-000904 868, ACH-000444 974, ACH-001249 1756, ACH-000854 899, ACH-002204 1318, ACH-000901 816, ACH-001230 947, ACH-001150 782, ACH-000923 1469, ACH-000960 913, ACH-002335 1312, ACH-000550 974, ACH-001955 1296, ACH-000258 872, ACH-000167 838, ACH-000593 764, ACH-000064 1203, ACH-000488 954, ACH-000837 1015, ACH-001656 902, ACH-001088 1337, ACH-001036 858, ACH-001079 2586, ACH-001171 792, ACH-000454 1051, ACH-000419 826, ACH-000118 794, ACH-000870 1557, ACH-000128 767, ACH-001957 1426, ACH-001239 851, ACH-000068 812, ACH-001043 825, ACH-001113 1072, ACH-000774 953, ACH-000868 1422, ACH-000300 1431, ACH-000842 929, ACH-000578 869, ACH-001045 822, ACH-001214 889, ACH-000044 1202, ACH-000600 939, ACH-000659 1129, ACH-000865 1358, ACH-000941 813, ACH-000509 873, ACH-001094 1036, ACH-000216 925, ACH-001225 792, ACH-001234 819, ACH-000710 968, ACH-001071 1175, ACH-000452 816, ACH-000887 1408, ACH-000195 2029, ACH-001956 1368, ACH-000327 819, ACH-000658 927, ACH-000690 771, ACH-000738 1064, ACH-000090 1024, ACH-001101 1005, ACH-000356 1294, ACH-000961 763, ACH-000028 868, ACH-000071 1287, ACH-000635 1368, ACH-001000 980, ACH-000836 1001]
 - Genes having a similar CN value accross all []


Gene level CN data:

__Rows__: DepMap cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

Segmented data:

__Columns__: DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean""")

## We push internal dataset with blacklisted removed

and we add it to eternal dataset and to virtual dataset

In [ ]:
## for segment removing first blacklisted, then embargoed, to create two datasets
segmentcn[~segmentcn.DepMap_ID.isin(blacklist)].to_csv('temp/internal_'+release+'_segs_cn')
genecn[~genecn.index.isin(blacklist)].to_csv('temp/internal_'+release+'_gene_cn')

In [ ]:
tc.update_dataset(dataset_permaname="depmap-wes-cn-data-81a7", 
                  upload_file_path_dict={
                    'temp/internal_'+release+'_gene_cn': 'NumericMatrixCSV',
                    'temp/internal_'+release+'_segs_cn': 'TableCSV'},
                  dataset_description=
"""
# Copy Number


## ** Version 1 Internal 18Q1****

Generated with the following script:

```
wes_pri <- taigr::load.from.taiga(data.name='gene-level-cn-87aa', 
                                  data.version=5, 
                                  data.file='gene_CN_WES_priority')
source_info <- data.frame(ccle_name=gsub("snp_|sangerWES_|ccleWES_|achillesWES_", 
                                         "", row.names(wes_pri)), 
                          source=gsub("_.*", "", row.names(wes_pri)))
wes_pri %<>% magrittr::set_rownames(source_info$ccle_name)

```

## ** Version 2 Internal 18Q2****

Generated with the following script:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=9, data.file='wes_priority_cn_gene_matrix') %>% log2()


```

## ** Version 3 Internal 18Q2****

Generated with the following script:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=11, data.file='wes_priority_cn_gene_matrix') %>% log2()


```
## ** Version 4-6 Internal 18Q3****

__Description__: log2 gene level copy number data

Generated with the following script:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=15, data.file='wes_priority_cn_gene_matrix') %>% log2()


```
__Rows__: Broad (arxspan) cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

Version 5 has updated cell line name mapping

Version 4 and 5 the segmented CN for Sanger's data is off by a factor of 2, version 6 corrects this

**** Version 7 Internal 18Q4****

__Description__: log2 gene level copy number data

Generated with the following script:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=17, data.file='wes_priority_cn_gene_matrix') %>% log2()


```
__Rows__: DepMap (arxspan) cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

## ** Version 8-9 Internal 19Q1****

version 9 has the correct data for 19Q1

## ** Version 10-11 Internal 19Q2****

__version 11 added an additional 13 cell lines and adds the segmented copy number data__

## ** Version 12 Internal 19Q3****

__Description__: log2(X + 1) gene level copy number data (data is now log2 transformed with a __pseudocount of 1__ added). CN data is generated using __hg38__. 


## ** Version 13 Internal 19Q4****

Adding 17 new cell lines, 3 reprioritized cell lines. log2(COPY RATIO+1). 
Some cells lines have been flagged as:

 - having bad looking copy ration plots = 
 - having too many segments = [ ACH-000458 762, ACH-000848 1171, ACH-001017 1223, ACH-000904 868, ACH-000444 974, ACH-001249 1756, ACH-000854 899, ACH-002204 1318, ACH-000901 816, ACH-001230 947, ACH-001150 782, ACH-000923 1469, ACH-000960 913, ACH-002335 1312, ACH-000550 974, ACH-001955 1296, ACH-000258 872, ACH-000167 838, ACH-000593 764, ACH-000064 1203, ACH-000488 954, ACH-000837 1015, ACH-001656 902, ACH-001088 1337, ACH-001036 858, ACH-001079 2586, ACH-001171 792, ACH-000454 1051, ACH-000419 826, ACH-000118 794, ACH-000870 1557, ACH-000128 767, ACH-001957 1426, ACH-001239 851, ACH-000068 812, ACH-001043 825, ACH-001113 1072, ACH-000774 953, ACH-000868 1422, ACH-000300 1431, ACH-000842 929, ACH-000578 869, ACH-001045 822, ACH-001214 889, ACH-000044 1202, ACH-000600 939, ACH-000659 1129, ACH-000865 1358, ACH-000941 813, ACH-000509 873, ACH-001094 1036, ACH-000216 925, ACH-001225 792, ACH-001234 819, ACH-000710 968, ACH-001071 1175, ACH-000452 816, ACH-000887 1408, ACH-000195 2029, ACH-001956 1368, ACH-000327 819, ACH-000658 927, ACH-000690 771, ACH-000738 1064, ACH-000090 1024, ACH-001101 1005, ACH-000356 1294, ACH-000961 763, ACH-000028 868, ACH-000071 1287, ACH-000635 1368, ACH-001000 980, ACH-000836 1001]
 - Genes having a similar CN value accross all []


Gene level CN data:

__Rows__: DepMap cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

Segmented data:

__Columns__: DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean
""")

In [ ]:
# To add to a virtual dataset
AddToVirtual(virtual_internal, 'depmap-wes-cn-data-81a7', [('CCLE_gene_cn', 'internal_'+release+'_gene_cn'),('CCLE_segment_cn', 'internal_'+release+'_segs_cn')])
# To add to a eternal dataset
AddToVirtual('depmap-a0ab', 'depmap-wes-cn-data-81a7', [('CCLE_gene_cn', 'internal_'+release+'_gene_cn'),('CCLE_segment_cn', 'internal_'+release+'_segs_cn')])

## We add to dmc as in internal

In [ ]:
os.system('mv temp/internal_'+release+'_gene_cn temp/dmc_'+release+'_gene_cn')
os.system('mv temp/internal_'+release+'_segs_cn temp/dmc_'+release+'_segs_cn')

In [ ]:
tc.update_dataset(dataset_permaname="depmap-cn-data-9b9d",
                upload_file_path_dict={
                    'temp/dmc_'+release+'_gene_cn':'NumericMatrixCSV',
                    'temp/dmc_'+release+'_segs_cn': 'TableCSV'
                   },
                
                  dataset_description="""
**** Version 1-2 DMC 19Q1****

version 2 contains the correct data for 19Q1

**** Version 3-4 DMC 19Q2****

__version 4 added an additional 13 cell lines and adds the segmented copy number data__

**** Version 5 DMC 19Q3***

__Description__: log2(X + 1) gene level copy number data (data is now log2 transformed with a __pseudocount of 1__ added). CN data is generated using __hg38__.  The segmented copy number data includes the mean segmented copy number segments.

Gene level CN data:

__Rows__: DepMap cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

Segmented data:

__Columns__: DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean
""")
AddToVirtual(virtual_dmc, "depmap-cn-data-9b9d", new_aliases=[('CCLE_gene_cn', 'internal_'+release+'_gene_cn'),('CCLE_segment_cn', 'internal_'+release+'_segs_cn')])

## We add to public as internal minus dmc embargoed and only cell lines from previous previous release (6 month)

In [ ]:
(set(wes_dmc_embargo) | set(blacklist))

In [ ]:
genecn[~genecn.index.isin(set(wes_dmc_embargo) | set(blacklist))]

In [ ]:
a = segmentcn[segmentcn.DepMap_ID.isin(prevprev)]
a[~a.DepMap_ID.isin(set(wes_dmc_embargo) | set(blacklist))].to_csv('temp/public_' + release + '_segs_cn')
a = genecn[genecn.index.isin(prevprev)]
a[~a.index.isin(set(wes_dmc_embargo) | set(blacklist))].to_csv('temp/public_'+release+'_gene_cn')

In [ ]:
tc.update_dataset(dataset_permaname='depmap-wes-cn-data-97cc',
                    upload_file_path_dict={
                    'temp/public_'+release+'_gene_cn':'NumericMatrixCSV',
                    'temp/public_'+release+'_segs_cn': 'TableCSV'
                   },
                  dataset_description="""
**** Versions 1-5 Public 18Q1****

Gene-level WES copy-number data for publicly accessible CCLE data. 

```

internal_lines <- readr::read_csv("~/Downloads/avana-broad-18q1_v2-sample-info.csv")$cell_line
public_lines <- readr::read_csv("~/Downloads/avana-public-tentative-18q1_v5-sample-info.csv")$cell_line
non_public_lines <- setdiff(internal_lines, public_lines)

full_cn_set <- taigr::load.from.taiga(data.name='gene-level-cn-87aa', data.version=5, data.file='full_gene_CN')
source_info <- data.frame(source=gsub("_.*", "", row.names(full_cn_set)),
                          ccle_name=gsub("snp_|achillesWES_|ccleWES_|sangerWES_", "",
                                         row.names(full_cn_set)),
                          row_idx=1:nrow(full_cn_set))
to_remove <- source_info %>%
  dplyr::filter(ccle_name %in% non_public_lines,
                source %in% c("ccleWES", "achillesWES"))
also_to_remove <- source_info %>%
                    dplyr::filter(source == "sangerWES")
indices_to_remove <- c(to_remove$row_idx, also_to_remove$row_idx) %>% unique()
indices_to_keep <- source_info %>%
  dplyr::filter(!(row_idx %in% indices_to_remove)) %>%
  dplyr::group_by(ccle_name) %>%
  dplyr::mutate(priority=ifelse(source == "snp", 4,
                                ifelse(source == "sangerWES", 3,
                                       ifelse(source == "ccleWES", 2, 1)))) %>%
  dplyr::filter(priority == min(priority)) %>%
  dplyr::ungroup()

public_cn <- full_cn_set[indices_to_keep$row_idx,]
source_info <- data.frame(source=gsub("_.*", "", row.names(public_cn)),
                          ccle_name=gsub("snp_|achillesWES_|ccleWES_|sangerWES_", "",
                                         row.names(public_cn)))
public_cn %<>% magrittr::set_rownames(source_info$ccle_name)
```

CN data are on a log2 scale.

`WES_source_info` tracks the source data for each cell line. Sources are `snp`, `achillesWES`, `ccleWES`, and `sangerWES`

NOTE: Version 1 contained WES data from cell lines not available in the 18Q1 Public release. Versions 2-4 contained Sanger's WES CN data

**** Version 6 Public 18Q2****

Generated by running:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=10, data.file='public_wes_priority_cn_gene_matrix') %>% log2()
```

**** Version 7 Public 18Q2****

Generated by running:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=11, data.file='public_wes_priority_cn_gene_matrix') %>% log2()
```


**** Version 8-9 Public 18Q3****

Generated by running:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=15, data.file='public_wes_priority_cn_gene_matrix') %>% log2()
```
Version 8 the segmented CN for Sanger's data is off by a factor of 2, version 9 corrects this

includes cell lines that should not be public

**** Version 10 Public 18Q1, 18Q2, 18Q3****

__use version 10 for 18Q1, 18Q2 and 18Q3 datasets__ 

Version 10 is the most up-to-date version of "public\_18Q3\_gene\_cn.csv". The three datasets have been updated to remove cell lines that should not have been made public. They are named in the portal and google bucket for portal downloads as v2, e.g. public\_18Q3\_gene\_cn\_v2.csv.

__Rows__: Broad (arxspan) cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

**** Version 11 Public 18Q4****

Generated by running:

```
wes_pri <- taigr::load.from.taiga(data.name='segmented-cn-wes-prioritzed-7fe1', data.version=17, data.file='public_wes_priority_cn_gene_matrix') %>% log2()
```

__Rows__: DepMap (arxspan) cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

**** Version 12-14 Public 19Q1****

version 14 contains the correct data for 19Q1

version 13 is the same as v12 except that it uses the original hg19 coordinates not hg38. The Achilles public data set uses the hg19 coordinates. 

**** Version 15-16 Public 19Q2****

__version 16 also adds the segmented copy number data__

**** Version 17-18 Public 19Q3****

__data is hg38 liftover__

__Description__: log2 + 1 gene level copy number data (data is log2 transformed with a __pseudocount of 1__ added). It uses hg19 coordinates. Also the segmented copy number data.

Gene level CN data:

__Rows__: DepMap cell line IDs

__Columns__: gene names in the format HGNC\_symbol (Entrez\_ID)

Segmented data:

__Columns__: DepMap\_ID, Chromosome, Start, End, Num\_Probes, Segment\_Mean, CCLE\_name

# /!\ Deprecated

This data version has been deprecated. It was used internally at the broad from 19Q1 to 19Q2.
It appeared to contain too many NaN values and was thus deprecated for a more recent dataset version.
We subseted this version with only the cell line listed for this public release.

**** Version 19 Public 19Q3 ****
""")
# To add to a virtual dataset
AddToVirtual(virtual_public, "depmap-wes-cn-data-97cc", new_aliases=[('CCLE_gene_cn', 'public_'+release+'_gene_cn'),('CCLE_segment_cn', 'public_'+release+'_segs_cn')])